In [1]:
import pandas as pd
import nltk
train1 = pd.read_csv(r"C:\Users\Admin\Downloads\spooky\train.csv")#19579
test = pd.read_csv(r"C:\Users\Admin\Downloads\spooky\test.csv")#8392
submit = pd.read_csv(r"C:\Users\Admin\Downloads\spooky\sample_submission.csv")

In [2]:
from math import floor
valid= train1.loc[:floor((train1.shape[0]*1)/3)]
train = train1.loc[floor((train1.shape[0]*1)/3):]
#train.head()


In [3]:
print train.shape[0]

13053


In [4]:
#test.head()

In [5]:
import re
from nltk.corpus import stopwords 
import string
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stop = set(stopwords.words('english'))# such as a, an, and, are,
exclude = set(string.punctuation) # such as: , . "" {}

#tokenize text
def token(doc):
    split = " ".join(i for i in doc.lower().split())
    words = re.findall(r"[\w']+", split)
    stop_free = " ".join([i for i in words if i not in stop]) # no stop word
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude) # no punctuation 
    token_a = re.findall(r"[\w']+", punc_free)
    token_b =[ps.stem(a) for a in token_a]
    token_c = " ".join([b for b in token_b])
    return token_c
train["token"] = [token(a) for a in train["text"]]
valid["token"]= [token(a) for a in valid["text"]]
test["token"] = [token(a) for a in test["text"]]
    

C:\Users\Admin\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Admin\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#dividing the training set into groups
hpl = train[train["author"] == "HPL"][['id','token']]
mws = train[train["author"] == "MWS"][['id','token']]
eap = train[train["author"] == "eap"][['id','token']]

In [7]:
text = list(train['text'])
print len(text)

13053


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vector= CountVectorizer()
X_train_counts = vector.fit_transform(list(train['text']))
X_train_counts.shape#2*8

(13053, 21449)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_tfidf = tfidf.fit_transform(X_train_counts)
print X_tfidf.shape

(13053, 21449)


In [10]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_tfidf, list(train['author']))

In [15]:
import numpy as np
test_vector = vector.fit_transform(list(valid['text']))
predicted = clf.predict(test_vector)
np.mean(predicted == valid['author'])

ValueError: dimension mismatch

# Naive Bayes

In [ ]:
#We firstly need to select the feature for each class ny Naive Bayes. 
#Based on those features, we can classify the valid set into groups

In [ ]:
list(valid['text'])